使用instructor进行知识抽取

In [2]:
from py2neo import Graph, Node, Relationship
import time

class NSTIKnowledgeGraph:
    def __init__(self, uri, user, password):
        """
        初始化Neo4j连接
        """
        self.graph = Graph(uri, auth=(user, password))
        self.graph.run("MATCH (n) DETACH DELETE n")  # 清空数据库（生产环境慎用）
        print("已连接到Neo4j数据库")
    
    def create_nodes_and_relationships(self):
        """
        创建所有节点和关系
        """
        print("开始创建知识图谱...")
        
        # 创建文献来源节点
        literature = Node("LiteratureSource", 
                         name="坏死性软组织感染临床诊治急诊专家共识",
                         description="中国医师协会急诊医师分会等组织制定的专家共识，提供NSTIs诊疗指南。")
        self.graph.create(literature)
        print("已创建文献来源节点")
        
        # 创建疾病节点
        diseases = [
            ("坏死性软组织感染", "由病原微生物感染导致皮下组织、筋膜或肌肉坏死的一类疾病，起病急、病情进展快，致死致残率高。"),
            ("坏死性筋膜炎", "NSTIs的一种类型，主要累及筋膜，常见于肢体和会阴部。"),
            ("Fournier坏疽", "发生于会阴、外生殖器、肛门等部位的NSTIs，起病隐匿、进展迅速。"),
            ("Meleney协同坏疽", "手术后进行性协同坏疽，常发于胸、腹手术或引流后，皮损呈现三区特征。"),
            ("眼眶周围NSTIs", "累及眼眶周围的NSTIs，可导致失明、死亡，表现为眼部疼痛、肿胀突出等。"),
            ("颈部NSTIs", "颈部筋膜和皮下组织的广泛坏死，多为混合感染，易下行扩散。"),
            ("肢体NSTIs", "发生于肢体的NSTIs，下肢多于上肢，常见于外伤或手术患者。"),
            ("胸腹部NSTIs", "累及胸腹部的NSTIs，如Meleney协同坏疽。"),
            ("会阴肛周NSTIs", "累及会阴和肛周的NSTIs，如Fournier坏疽。")
        ]
        
        disease_nodes = {}
        for name, desc in diseases:
            node = Node("Disease", name=name, description=desc)
            self.graph.create(node)
            disease_nodes[name] = node
            # 添加文献来源关系
            rel = Relationship(node, "SOURCE_FROM", literature)
            self.graph.create(rel)
        
        print(f"已创建{len(diseases)}个疾病节点")
        
        # 创建症状节点
        symptoms = [
            ("肿胀", "局部皮肤肿胀，是NSTIs的早期表现之一。"),
            ("疼痛", "局部疼痛，早期可能与皮肤损伤程度不一致，后期转为麻木。"),
            ("红斑", "皮肤红斑，是NSTIs的常见早期症状。"),
            ("发热", "全身发热，常见于NSTIs患者。"),
            ("血泡", "皮肤出现血泡，提示病情严重，易伴休克和截肢。"),
            ("瘀斑", "皮肤大片瘀斑，表明组织缺血性坏死。"),
            ("皮肤发黑", "皮肤发黑或呈暗红色，提示组织坏死。"),
            ("皮下捻发音", "皮下组织有捻发音，表明气体积累，常见于产气菌感染。"),
            ("皮肤坏死破溃", "皮肤坏死并破溃，是NSTIs的晚期表现。"),
            ("心动过速", "心率加快，常见于全身感染反应。"),
            ("低血压", "血压降低，可能进展为休克。"),
            ("休克", "脓毒性休克，是重症NSTIs的表现。"),
            ("呼吸急促", "呼吸频率增加，可能表明呼吸衰竭。"),
            ("少尿", "尿量减少，提示急性肾功能损伤。"),
            ("无尿", "无尿输出，表明严重肾功能衰竭。"),
            ("黄疸", "皮肤和巩膜黄染，提示肝功能损伤。"),
            ("眼部疼痛", "眼眶周围NSTIs的症状，伴视力下降和眼肌麻痹。"),
            ("吞咽困难", "颈部NSTIs的症状，因水肿导致气道梗阻。")
        ]
        
        symptom_nodes = {}
        for name, desc in symptoms:
            node = Node("Symptom", name=name, description=desc)
            self.graph.create(node)
            symptom_nodes[name] = node
        
        print(f"已创建{len(symptoms)}个症状节点")
        
        # 创建检查方法节点
        tests = [
            ("LRINEC评分", "实验室风险指标用于坏死性筋膜炎评估，评分≥6分提示可能NSTIs。"),
            ("血清肌酸激酶", "CK水平升高，可用于NSTIs的辅助诊断。"),
            ("血栓调节蛋白", "TM水平升高，对NSTIs和非坏死感染有区分能力。"),
            ("MRI", "磁共振成像，显示深筋膜增厚和异常信号，用于NSTIs诊断。"),
            ("CT", "计算机断层扫描，显示皮下气体影和筋膜水肿，用于深部感染诊断。"),
            ("床旁超声", "超声检查，快速、无创，显示水肿和积气，用于初步诊断。"),
            ("手术探查", "诊断NSTIs的金标准，通过手指试验和直视判断坏死组织。"),
            ("组织学检查", "病理学检查，显示筋膜坏死和炎症，辅助诊断。"),
            ("手指试验", "手术探查的一部分，切开深筋膜观察组织分离和渗出。"),
            ("血培养", "血液样本培养，用于病原体鉴定。"),
            ("病灶处组织液培养", "感染部位组织液培养，用于病原体鉴定。"),
            ("坏死组织培养", "坏死组织样本培养，用于病原体鉴定。"),
            ("宏基因组学", "mNGS技术，用于病原体基因检测。")
        ]
        
        test_nodes = {}
        for name, desc in tests:
            node = Node("Test", name=name, description=desc)
            self.graph.create(node)
            test_nodes[name] = node
            # 为关键检查方法添加文献来源关系
            if name in ["LRINEC评分", "手术探查"]:
                rel = Relationship(node, "SOURCE_FROM", literature)
                self.graph.create(rel)
        
        print(f"已创建{len(tests)}个检查方法节点")
        
        # 创建治疗方法节点
        treatments = [
            ("抗感染治疗", "使用抗菌药物控制感染，应早期、足量、联合用药。"),
            ("手术治疗", "早期切开减张和清创，是NSTIs的核心治疗。"),
            ("清创术", "彻底清除坏死组织，直至健康组织，改善预后。"),
            ("截肢", "对于肢体严重坏死且感染快速进展者，需截肢保命。"),
            ("器官支持治疗", "包括液体复苏、呼吸支持等，维护器官功能。"),
            ("高压氧治疗", "辅助治疗，提高血氧分压，抑制厌氧菌，但疗效有争议。"),
            ("静脉输注免疫球蛋白", "辅助治疗，中和超抗原，减少炎症，但证据有限。"),
            ("中医中药治疗", "中医辨证治疗，如清热解毒、活血化瘀等。"),
            ("呼吸支持", "机械通气或氧疗，用于呼吸衰竭患者。"),
            ("液体复苏", "补液维持血压和组织灌注，用于休克患者。"),
            ("血管活性药物", "如去甲肾上腺素，用于维持血压。"),
            ("血液净化", "用于肾功能衰竭或清除炎症因子。")
        ]
        
        treatment_nodes = {}
        for name, desc in treatments:
            node = Node("Treatment", name=name, description=desc)
            self.graph.create(node)
            treatment_nodes[name] = node
            # 为关键治疗方法添加文献来源关系
            if name in ["抗感染治疗", "手术治疗"]:
                rel = Relationship(node, "SOURCE_FROM", literature)
                self.graph.create(rel)
        
        print(f"已创建{len(treatments)}个治疗方法节点")
        
        # 创建病原体节点
        pathogens = [
            ("A族溶血性链球菌", "GAS，常见于Ⅱ型NSTIs，引起快速进展感染。"),
            ("耐甲氧西林金黄色葡萄球菌", "MRSA，常见于Ⅱ型NSTIs，与外伤或手术相关。"),
            ("创伤弧菌", "海洋菌群，引起Ⅲ型NSTIs，多见于沿海地区。"),
            ("嗜水气单胞菌", "水生菌，引起Ⅲ型NSTIs，常见于接触海水。"),
            ("真菌", "引起Ⅳ型NSTIs，多见于免疫功能低下患者。"),
            ("混合病原体", "需氧菌和厌氧菌混合，引起Ⅰ型NSTIs，最常见。"),
            ("产气荚膜梭菌", "厌氧菌，见于皮肤皮损或严重穿透伤。"),
            ("溶组织梭菌", "厌氧菌，见于皮肤皮损或严重穿透伤。")
        ]
        
        pathogen_nodes = {}
        for name, desc in pathogens:
            node = Node("Pathogen", name=name, description=desc)
            self.graph.create(node)
            pathogen_nodes[name] = node
        
        print(f"已创建{len(pathogens)}个病原体节点")
        
        # 创建风险因素节点
        risk_factors = [
            ("高龄", "年龄≥60岁，是NSTIs的易患因素。"),
            ("酗酒", "酒精滥用，增加NSTIs风险。"),
            ("肥胖", "肥胖患者更易发生NSTIs。"),
            ("糖尿病", "糖尿病患者免疫功能受损，增加NSTIs风险。"),
            ("慢性肝病", "肝功能不全，增加NSTIs风险。"),
            ("慢性肾病", "肾功能不全，增加NSTIs风险。"),
            ("使用免疫抑制剂", "免疫抑制治疗，增加NSTIs风险。"),
            ("免疫功能低下", "各种原因导致的免疫缺陷，增加NSTIs风险。"),
            ("长期卧床", "活动受限，增加皮肤软组织感染风险。"),
            ("皮肤软组织损伤", "如挤压伤、肌肉拉伤，是NSTIs常见诱因。"),
            ("生食海产品", "增加创伤弧菌感染风险。"),
            ("接触带菌海水", "增加海洋菌群感染风险。")
        ]
        
        risk_factor_nodes = {}
        for name, desc in risk_factors:
            node = Node("RiskFactor", name=name, description=desc)
            self.graph.create(node)
            risk_factor_nodes[name] = node
        
        print(f"已创建{len(risk_factors)}个风险因素节点")
        
        # 创建鉴别诊断节点
        differential_diagnoses = [
            ("深静脉血栓形成", "急性下肢深静脉血栓，表现为患肢肿胀、疼痛、水肿等，静脉超声可明确。"),
            ("动脉栓塞", "动脉阻塞，出现'5P'征（疼痛、麻痹、苍白、无脉、感觉异常），血管超声可明确。"),
            ("隐翅虫皮炎", "隐翅虫体液毒素引起的皮肤炎症，表现为红斑、肿痛、脓疱等。"),
            ("其他炎性或非炎性的皮炎", "其他皮肤炎症性疾病，需与NSTIs鉴别。")
        ]
        
        diff_diag_nodes = {}
        for name, desc in differential_diagnoses:
            node = Node("DifferentialDiagnosis", name=name, description=desc)
            self.graph.create(node)
            diff_diag_nodes[name] = node
        
        print(f"已创建{len(differential_diagnoses)}个鉴别诊断节点")
        
        # 创建主要疾病与症状的关系
        main_disease = disease_nodes["坏死性软组织感染"]
        symptom_names = ["肿胀", "疼痛", "红斑", "发热", "血泡", "瘀斑", "皮肤发黑", "皮下捻发音", "皮肤坏死破溃", "休克"]
        for name in symptom_names:
            if name in symptom_nodes:
                rel = Relationship(main_disease, "HAS_SYMPTOM", symptom_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(symptom_names)}个疾病-症状关系")
        
        # 创建疾病与检查方法的关系
        test_names = ["LRINEC评分", "血清肌酸激酶", "血栓调节蛋白", "MRI", "CT", "床旁超声", "手术探查"]
        for name in test_names:
            if name in test_nodes:
                rel = Relationship(main_disease, "DIAGNOSED_BY", test_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(test_names)}个疾病-检查方法关系")
        
        # 创建疾病与治疗方法的关系
        treatment_names = ["抗感染治疗", "手术治疗", "清创术", "器官支持治疗"]
        for name in treatment_names:
            if name in treatment_nodes:
                rel = Relationship(main_disease, "TREATED_WITH", treatment_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(treatment_names)}个疾病-治疗方法关系")
        
        # 创建疾病与病原体的关系
        pathogen_names = ["A族溶血性链球菌", "耐甲氧西林金黄色葡萄球菌", "创伤弧菌", "混合病原体"]
        for name in pathogen_names:
            if name in pathogen_nodes:
                rel = Relationship(main_disease, "CAUSED_BY", pathogen_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(pathogen_names)}个疾病-病原体关系")
        
        # 创建疾病与风险因素的关系
        risk_factor_names = ["高龄", "糖尿病", "慢性肝病", "皮肤软组织损伤"]
        for name in risk_factor_names:
            if name in risk_factor_nodes:
                rel = Relationship(main_disease, "HAS_RISK_FACTOR", risk_factor_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(risk_factor_names)}个疾病-风险因素关系")
        
        # 创建疾病与鉴别诊断的关系
        for name in diff_diag_nodes:
            rel = Relationship(main_disease, "DIFFERENTIAL_DIAGNOSIS", diff_diag_nodes[name])
            self.graph.create(rel)
        
        print(f"已创建{len(diff_diag_nodes)}个疾病-鉴别诊断关系")
        
        print("知识图谱创建完成！")

def main():
    """
    主函数
    """
    # Neo4j数据库连接信息 - 请根据您的实际配置修改
    NEO4J_URI = "bolt://localhost:7687"  # 或您的Neo4j服务器地址
    NEO4J_USER = "neo4j"                 # 默认用户名
    NEO4J_PASSWORD = "test1234"          # 您的Neo4j密码
    
    try:
        # 创建知识图谱实例
        kg = NSTIKnowledgeGraph(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
        
        # 创建节点和关系
        start_time = time.time()
        kg.create_nodes_and_relationships()
        end_time = time.time()
        
        print(f"知识图谱构建完成，耗时: {end_time - start_time:.2f}秒")
        
        # 执行一个示例查询
        result = kg.graph.run("""
        MATCH (d:Disease {name: '坏死性软组织感染'})-[:HAS_SYMPTOM]->(s:Symptom)
        RETURN d.name as disease, COLLECT(s.name) as symptoms
        """).data()
        
        print("\n示例查询结果:")
        for record in result:
            print(f"疾病: {record['disease']}")
            print(f"症状: {', '.join(record['symptoms'])}")
            
    except Exception as e:
        print(f"错误: {e}")
        print("请检查Neo4j数据库连接设置")

if __name__ == "__main__":
    main()

已连接到Neo4j数据库
开始创建知识图谱...
已创建文献来源节点
已创建9个疾病节点
已创建18个症状节点
已创建13个检查方法节点
已创建12个治疗方法节点
已创建8个病原体节点
已创建12个风险因素节点
已创建4个鉴别诊断节点
已创建10个疾病-症状关系
已创建7个疾病-检查方法关系
已创建4个疾病-治疗方法关系
已创建4个疾病-病原体关系
已创建4个疾病-风险因素关系
已创建4个疾病-鉴别诊断关系
知识图谱创建完成！
知识图谱构建完成，耗时: 3.08秒

示例查询结果:
疾病: 坏死性软组织感染
症状: 休克, 皮肤坏死破溃, 皮下捻发音, 皮肤发黑, 瘀斑, 血泡, 发热, 红斑, 疼痛, 肿胀


添加新的文档的节点

In [3]:
from py2neo import Graph, Node, Relationship
import time

class KnowledgeGraphExtender:
    def __init__(self, uri, user, password):
        """
        初始化Neo4j连接
        """
        self.graph = Graph(uri, auth=(user, password))
        print("已连接到Neo4j数据库")
    
    def add_asthma_nodes_and_relationships(self):
        """
        添加支气管哮喘相关的节点和关系，使用原有的节点和关系类型
        """
        print("开始添加支气管哮喘相关节点...")
        
        # 创建文献来源节点（使用原有的LiteratureSource类型）
        asthma_literature = Node("LiteratureSource", 
                               name="支气管哮喘急性发作评估及处理中国专家共识",
                               description="中华医学会呼吸病学分会哮喘学组制定的专家共识")
        self.graph.create(asthma_literature)
        print("已创建哮喘文献来源节点")
        
        # 创建疾病节点（使用原有的Disease类型）
        asthma_diseases = [
            ("支气管哮喘", "慢性气道炎症性疾病，特征为可逆性气流受限和气道高反应性。"),
            ("支气管哮喘急性发作", "喘息、气促、咳嗽、胸闷等症状突然发生或急剧加重。"),
            ("危重症哮喘", "哮喘急性发作的最严重形式，可危及生命。")
        ]
        
        asthma_disease_nodes = {}
        for name, desc in asthma_diseases:
            node = Node("Disease", name=name, description=desc)
            self.graph.create(node)
            asthma_disease_nodes[name] = node
            # 添加文献来源关系（使用原有的SOURCE_FROM关系）
            rel = Relationship(node, "SOURCE_FROM", asthma_literature)
            self.graph.create(rel)
        
        print(f"已创建{len(asthma_diseases)}个哮喘疾病节点")
        
        # 创建症状节点（使用原有的Symptom类型）
        asthma_symptoms = [
            ("喘息", "呼吸时发出的哨笛音，是哮喘的典型症状。"),
            ("气促", "呼吸急促、困难，感觉气不够用。"),
            ("咳嗽", "持续性或发作性咳嗽，夜间和清晨加重。"),
            ("胸闷", "胸部压迫感或紧缩感。"),
            ("呼吸困难", "呼吸费力，严重时出现三凹征。"),
            ("端坐呼吸", "严重呼吸困难时需要端坐位呼吸。"),
            ("讲话困难", "严重时只能说单词或单字。")
        ]
        
        asthma_symptom_nodes = {}
        for name, desc in asthma_symptoms:
            node = Node("Symptom", name=name, description=desc)
            self.graph.create(node)
            asthma_symptom_nodes[name] = node
        
        print(f"已创建{len(asthma_symptoms)}个哮喘症状节点")
        
        # 创建检查方法节点（使用原有的Test类型）
        asthma_tests = [
            ("肺功能检查", "评估气流受限程度，包括FEV1、PEF等指标。"),
            ("呼气峰流速", "PEF，简单有效的肺功能监测方法。"),
            ("动脉血气分析", "评估氧合和通气状况，判断呼吸衰竭类型。"),
            ("血嗜酸性粒细胞计数", "哮喘炎症的生物标志物。")
        ]
        
        asthma_test_nodes = {}
        for name, desc in asthma_tests:
            node = Node("Test", name=name, description=desc)
            self.graph.create(node)
            asthma_test_nodes[name] = node
        
        print(f"已创建{len(asthma_tests)}个哮喘检查方法节点")
        
        # 创建治疗方法节点（使用原有的Treatment类型）
        asthma_treatments = [
            ("短效β2受体激动剂", "SABA，快速缓解支气管痉挛。"),
            ("吸入性糖皮质激素", "ICS，控制气道炎症的基础药物。"),
            ("全身性糖皮质激素", "口服或静脉用激素，用于中重度急性发作。"),
            ("抗胆碱能药物", "如异丙托溴铵，与SABA联用增强支气管舒张效果。"),
            ("茶碱类药物", "支气管舒张剂，如氨茶碱。")
        ]
        
        asthma_treatment_nodes = {}
        for name, desc in asthma_treatments:
            node = Node("Treatment", name=name, description=desc)
            self.graph.create(node)
            asthma_treatment_nodes[name] = node
        
        print(f"已创建{len(asthma_treatments)}个哮喘治疗方法节点")
        
        # 创建风险因素节点（使用原有的RiskFactor类型）
        asthma_risk_factors = [
            ("呼吸道感染", "病毒或细菌感染是哮喘急性发作的主要诱因。"),
            ("过敏原吸入", "如尘螨、花粉、宠物皮屑等。"),
            ("吸烟", "主动或被动吸烟均可诱发和加重哮喘。"),
            ("空气污染", "室外和室内污染物。"),
            ("运动", "运动诱发的支气管痉挛。")
        ]
        
        asthma_risk_factor_nodes = {}
        for name, desc in asthma_risk_factors:
            node = Node("RiskFactor", name=name, description=desc)
            self.graph.create(node)
            asthma_risk_factor_nodes[name] = node
        
        print(f"已创建{len(asthma_risk_factors)}个哮喘风险因素节点")
        
        # 创建病原体节点（使用原有的Pathogen类型）
        asthma_pathogens = [
            ("鼻病毒", "导致哮喘发作的最常见病毒。"),
            ("流感病毒", "可诱发哮喘急性发作的病毒。"),
            ("呼吸道合胞病毒", "可诱发哮喘急性发作的病毒。")
        ]
        
        asthma_pathogen_nodes = {}
        for name, desc in asthma_pathogens:
            node = Node("Pathogen", name=name, description=desc)
            self.graph.create(node)
            asthma_pathogen_nodes[name] = node
        
        print(f"已创建{len(asthma_pathogens)}个哮喘病原体节点")
        
        # 创建鉴别诊断节点（使用原有的DifferentialDiagnosis类型）
        asthma_differential_diagnoses = [
            ("急性左心功能不全", "需要与哮喘急性发作鉴别的疾病。"),
            ("慢性阻塞性肺疾病急性加重", "需要与哮喘急性发作鉴别的疾病。"),
            ("急性肺栓塞", "需要与哮喘急性发作鉴别的疾病。"),
            ("上气道阻塞", "需要与哮喘急性发作鉴别的疾病。")
        ]
        
        asthma_diff_diag_nodes = {}
        for name, desc in asthma_differential_diagnoses:
            node = Node("DifferentialDiagnosis", name=name, description=desc)
            self.graph.create(node)
            asthma_diff_diag_nodes[name] = node
        
        print(f"已创建{len(asthma_differential_diagnoses)}个哮喘鉴别诊断节点")
        
        # 创建主要疾病与症状的关系（使用原有的HAS_SYMPTOM关系）
        main_asthma = asthma_disease_nodes["支气管哮喘急性发作"]
        asthma_symptom_names = ["喘息", "气促", "咳嗽", "胸闷", "呼吸困难"]
        for name in asthma_symptom_names:
            if name in asthma_symptom_nodes:
                rel = Relationship(main_asthma, "HAS_SYMPTOM", asthma_symptom_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(asthma_symptom_names)}个疾病-症状关系")
        
        # 创建疾病与检查方法的关系（使用原有的DIAGNOSED_BY关系）
        asthma_test_names = ["肺功能检查", "呼气峰流速", "动脉血气分析"]
        for name in asthma_test_names:
            if name in asthma_test_nodes:
                rel = Relationship(main_asthma, "DIAGNOSED_BY", asthma_test_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(asthma_test_names)}个疾病-检查方法关系")
        
        # 创建疾病与治疗方法的关系（使用原有的TREATED_WITH关系）
        asthma_treatment_names = ["短效β2受体激动剂", "吸入性糖皮质激素", "全身性糖皮质激素"]
        for name in asthma_treatment_names:
            if name in asthma_treatment_nodes:
                rel = Relationship(main_asthma, "TREATED_WITH", asthma_treatment_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(asthma_treatment_names)}个疾病-治疗方法关系")
        
        # 创建疾病与风险因素的关系（使用原有的HAS_RISK_FACTOR关系）
        asthma_risk_factor_names = ["呼吸道感染", "过敏原吸入", "吸烟"]
        for name in asthma_risk_factor_names:
            if name in asthma_risk_factor_nodes:
                rel = Relationship(main_asthma, "HAS_RISK_FACTOR", asthma_risk_factor_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(asthma_risk_factor_names)}个疾病-风险因素关系")
        
        # 创建疾病与病原体的关系（使用原有的CAUSED_BY关系）
        asthma_pathogen_names = ["鼻病毒", "流感病毒"]
        for name in asthma_pathogen_names:
            if name in asthma_pathogen_nodes:
                rel = Relationship(main_asthma, "CAUSED_BY", asthma_pathogen_nodes[name])
                self.graph.create(rel)
        
        print(f"已创建{len(asthma_pathogen_names)}个疾病-病原体关系")
        
        # 创建疾病与鉴别诊断的关系（使用原有的DIFFERENTIAL_DIAGNOSIS关系）
        for name in asthma_diff_diag_nodes:
            rel = Relationship(main_asthma, "DIFFERENTIAL_DIAGNOSIS", asthma_diff_diag_nodes[name])
            self.graph.create(rel)
        
        print(f"已创建{len(asthma_diff_diag_nodes)}个疾病-鉴别诊断关系")
        
        print("支气管哮喘相关节点添加完成！")
    
    def query_asthma_data(self):
        """
        查询哮喘相关数据的示例
        """
        print("\n查询哮喘相关数据示例:")
        
        # 查询哮喘疾病及其症状
        result1 = self.graph.run("""
        MATCH (d:Disease {name: '支气管哮喘急性发作'})-[:HAS_SYMPTOM]->(s:Symptom)
        RETURN d.name as disease, COLLECT(s.name) as symptoms
        """).data()
        
        for record in result1:
            print(f"疾病: {record['disease']}")
            print(f"症状: {', '.join(record['symptoms'])}")
        
        # 查询哮喘治疗方法
        result2 = self.graph.run("""
        MATCH (d:Disease {name: '支气管哮喘急性发作'})-[:TREATED_WITH]->(t:Treatment)
        RETURN d.name as disease, COLLECT(t.name) as treatments
        """).data()
        
        for record in result2:
            print(f"疾病: {record['disease']}")
            print(f"治疗方法: {', '.join(record['treatments'])}")

def main():
    """
    主函数
    """
    # Neo4j数据库连接信息 - 请根据您的实际配置修改
    NEO4J_URI = "bolt://localhost:7687"  # 或您的Neo4j服务器地址
    NEO4J_USER = "neo4j"                 # 默认用户名
    NEO4J_PASSWORD = "test1234"          # 您的Neo4j密码
    
    try:
        # 创建知识图谱扩展实例
        kg_extender = KnowledgeGraphExtender(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
        
        # 添加哮喘相关节点和关系
        start_time = time.time()
        kg_extender.add_asthma_nodes_and_relationships()
        end_time = time.time()
        
        print(f"哮喘知识图谱扩展完成，耗时: {end_time - start_time:.2f}秒")
        
        # 执行示例查询
        kg_extender.query_asthma_data()
            
    except Exception as e:
        print(f"错误: {e}")
        print("请检查Neo4j数据库连接设置")

if __name__ == "__main__":
    main()

已连接到Neo4j数据库
开始添加支气管哮喘相关节点...
已创建哮喘文献来源节点
已创建3个哮喘疾病节点
已创建7个哮喘症状节点
已创建4个哮喘检查方法节点
已创建5个哮喘治疗方法节点
已创建5个哮喘风险因素节点
已创建3个哮喘病原体节点
已创建4个哮喘鉴别诊断节点
已创建5个疾病-症状关系
已创建3个疾病-检查方法关系
已创建3个疾病-治疗方法关系
已创建3个疾病-风险因素关系
已创建2个疾病-病原体关系
已创建4个疾病-鉴别诊断关系
支气管哮喘相关节点添加完成！
哮喘知识图谱扩展完成，耗时: 0.89秒

查询哮喘相关数据示例:
疾病: 支气管哮喘急性发作
症状: 呼吸困难, 胸闷, 咳嗽, 气促, 喘息
疾病: 支气管哮喘急性发作
治疗方法: 全身性糖皮质激素, 吸入性糖皮质激素, 短效β2受体激动剂
